## Notebook Overview

This notebook allows you to explore and interact with a dataset of simulated LLM responses to survey questions. The dataset contains questions, background information about the simulated respondent, and the LLM's generated answer.

The key steps involved are:

1.  **Download the dataset:** The notebook downloads a JSONL file containing the preprocessed data from a Google Drive link.
2.  **Load and prepare data:** The data is loaded from the downloaded file and processed into a usable format.
3.  **Run sanity check:** This section allows you to interactively answer random questions from the dataset and compare your answer to the LLM's recorded response.

You can re-run the "Run sanity check" cell as often as you like to fetch new random questions.

In [1]:
!pip install -q gspread oauth2client
!pip install -q --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [2]:
import json
import os
import random
import sys
import textwrap
from pathlib import Path
from typing import Dict, List
from google.colab import auth
import gspread
from google.auth import default
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io

In [3]:

# Authenticate with Google Colab
auth.authenticate_user()

# Authorize gspread to access your Google Drive
# This creates a client to interact with the Google Drive API
scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
credentials, project_id = default()
client = gspread.authorize(credentials.with_scopes(scope))

print("✅ Google Sheets authentication successful.")

✅ Google Sheets authentication successful.


In [4]:
# This will pop up a window to ask for permission.
auth.authenticate_user()
drive_service = build('drive', 'v3')

# --- IMPORTANT: Change these two variables ---
FILE_URL = 'https://drive.google.com/file/d/1uH5FPiJjRZzjzcn_6lYYpCzsaJ69p1dl/view?usp=sharing'
OUTPUT_PATH = '/content/my_data.jsonl' # The path to save the file in Colab
# ---

# Step 3: Extract the File ID from the URL
# The ID is the long string of characters in the middle of the URL.
try:
    file_id = FILE_URL.split('/d/')[1].split('/')[0]
except IndexError:
    raise ValueError("Invalid Google Drive URL. Please use the full URL from the address bar.")

# Step 4: Download the file content
print(f"Downloading file with ID: {file_id}...")
request = drive_service.files().get_media(fileId=file_id)
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)

done = False
while not done:
    status, done = downloader.next_chunk()
    if status:
        print(f"Download {int(status.progress() * 100)}%.")

# Step 5: Write the downloaded content to the local file in Colab
with open(OUTPUT_PATH, 'wb') as f:
    f.write(fh.getvalue())

print(f"\n✅ File successfully downloaded to: {OUTPUT_PATH}")

# Optional: Verify the first few lines of the downloaded file
!head -n 5 {OUTPUT_PATH}

Download 87%.
Download 100%.

✅ File successfully downloaded to: /content/my_data.jsonl
{"source_file": "question_0/AfD-AfD/out_41-mini_v0_0.json", "model_prefix": "out_41-mini", "question_id": "intro", "question_content": "Auf einer Skala von 1 bis 7: Wie sehr stimmst du der Aussage zu: Auf allen Autobahnen soll ein generelles Tempolimit gelten. Antworte nur mit einer Zahl.", "iteration": 0, "prompt_messages": [{"role": "system", "content": "Scenario: Du diskutierst über die Aussage: Auf allen Autobahnen soll ein generelles Tempolimit gelten..\nBackground Story: Ich bin 59 Jahre alt und weiblich. Ich habe einen Hochschulabschluss, ein mittleres monatliches Haushalts-Nettoeinkommen und ich bin berufstätig. Ich bin überhaupt nicht religiös. Politisch-ideologisch ordne ich mich mittig rechts ein. Ich identifiziere mich sehr stark mit der Partei AfD und habe AfD gewählt. Ich lebe in Westdeutschland. Ich finde, die Regierung sollte die Einwanderung einschränken und habe keine Meinung dazu,

In [5]:
# Configure the location of the preprocessed dataset


DATASET_PATH = Path(OUTPUT_PATH)
if not DATASET_PATH.exists():
    raise FileNotFoundError(
        f"Dataset not found at {DATASET_PATH}. Upload or mount the JSONL file before proceeding."
    )

print(f"Using preprocessed dataset: {DATASET_PATH.resolve()}")

Using preprocessed dataset: /content/my_data.jsonl


In [6]:
def load_preprocessed_entries(dataset_path: Path) -> List[Dict[str, object]]:
    entries: List[Dict[str, object]] = []
    with dataset_path.open('r', encoding='utf-8') as fh:
        for line in fh:
            line = line.strip()
            if not line:
                continue
            entries.append(json.loads(line))
    if not entries:
        raise ValueError(f'No entries found in {dataset_path}')
    return entries


def pretty_print_prompt(prompt_messages: List[Dict[str, str]]) -> None:
    for message in prompt_messages:
        role = message.get('role', '?').upper()
        content = message.get('content', '').strip()
        wrapped = textwrap.fill(content, width=100, subsequent_indent='    ')
        print(f"{role}: {wrapped}")


def compare_answers(user_answer: str, model_answer: str) -> None:
    #print(f"\nYour answer:  {user_answer}")
    #print(f"LLM answer:   {model_answer}")
    try:
        user_val = float(user_answer)
        model_val = float(model_answer)
    except ValueError:
        return
    diff = abs(user_val - model_val)
    #print(f"Difference:   {diff:.2f}\n")


def run_sanity_check(
    entries: List[Dict[str, object]],
    sheet: gspread.Worksheet | None,
    n_rounds: int = 5,
    seed: int | None = None,
) -> None:
    if seed is not None:
        random.seed(seed)

    if sheet and not sheet.get_all_values():
        sheet.append_row([
            'Source File',
            'Question ID',
            'Question Content',
            'Iteration',
            'Respondent Name',
            'Respondent Type',
            'User Answer',
            'LLM Answer',
        ])

    for round_num in range(1, n_rounds + 1):
        entry = random.choice(entries)
        raw_prompts = entry.get('prompt_messages') or []
        prompt_messages: List[Dict[str, str]] = []
        if isinstance(raw_prompts, list):
            for item in raw_prompts:
                if isinstance(item, dict):
                    prompt_messages.append(
                        {
                            'role': str(item.get('role', '?')),
                            'content': str(item.get('content', '')),
                        }
                    )
        model_answer = str(entry.get('model_answer', '')).strip()

        print('=' * 80)
        print(f"Round {round_num}")
        source_file = entry.get('source_file', 'unknown')
        question_id = entry.get('question_id', 'unknown')
        question_content = entry.get('question_content', 'Unknown question')
        iteration = entry.get('iteration', 'n/a')
        entity = entry.get('entity')
        respondent_name = 'n/a'
        respondent_type = 'unknown'
        if isinstance(entity, dict):
            respondent_name = entity.get('name', 'n/a')
            respondent_type = entity.get('person_type', 'unknown')

        print('-- Prompt --')
        pretty_print_prompt(prompt_messages)

        user_answer = input('\nAssistant answer: ').strip()


        if user_answer == "":
            print("Skipping this round.")
            continue

        compare_answers(user_answer, model_answer)

        stringefied_prompt = json.dumps(prompt_messages)

        # Log data to Google Sheet if sheet is available
        if sheet:
            try:
                sheet.append_row([
                    source_file,
                    question_id,
                    question_content,
                    iteration,
                    respondent_name,
                    respondent_type,
                    user_answer,
                    model_answer,
                    stringefied_prompt,
                ])
            except Exception as e:
                print(f"Error appending row to Google Sheet: {e}")

    print('All rounds completed.')

In [ ]:

# Load dataset and launch the sanity check interaction
entries = load_preprocessed_entries(DATASET_PATH)
print(f"Loaded {len(entries)} entries from {DATASET_PATH}")

# Specify the Google Sheet URL
sheet_url = 'https://docs.google.com/spreadsheets/d/1CZduF8-mNQD0W8hPRNzmPH0kME1FAIofnwPFPmLne2k/edit?usp=sharing'

# Extract the sheet ID from the URL
try:
    sheet_id = sheet_url.split('/d/')[1].split('/')[0]
except IndexError:
    print("Error: Invalid Google Sheet URL.")
    sheet = None

# Open the Google Sheet by ID
if sheet_id:
    try:
        sheet = client.open_by_key(sheet_id).sheet1
        print(f"Connected to Google Sheet with ID: {sheet_id}")
    except Exception as e:
        print(f"Error opening Google Sheet: {e}")
        print("Proceeding without logging to Google Sheet.")
        sheet = None
else:
    sheet = None


run_sanity_check(entries, sheet=sheet, n_rounds=20)

Loaded 26208 entries from /content/my_data.jsonl
Connected to Google Sheet with ID: 1CZduF8-mNQD0W8hPRNzmPH0kME1FAIofnwPFPmLne2k
Round 1
-- Prompt --
SYSTEM: Scenario: Du diskutierst über die Aussage: Die Möglichkeiten der Vermieterinnen und Vermieter,
    Wohnungsmieten zu erhöhen, sollen gesetzlich stärker begrenzt werden.. Background Story: Ich bin
    48 Jahre alt und männlich. Ich habe einen Realschulabschluss, ein mittleres monatliches
    Haushalts-Nettoeinkommen und ich bin berufstätig. Ich bin nicht sehr religiös. Politisch-
    ideologisch ordne ich mich in der Mitte ein. Ich identifiziere mich ziemlich stark mit der
    Partei CDU/CSU und habe CDU/CSU gewählt. Ich lebe in Ostdeutschland. Ich finde, die Regierung
    sollte die Einwanderung einschränken und keine Maßnahmen ergreifen, um die
    Einkommensunterschiede zu verringern. The following is a debate between you and another person
USER: Auf einer Skala von 1 bis 7: Wie sehr stimmst du der Aussage zu: Die Möglichkeiten 